# EXAMEN II BIMESTRE

Elizabeth Silva

Requerimientos

In [70]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import pandas as pd
import re
from scipy.sparse import hstack
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from openai import OpenAI

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ELI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ELI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Preprocesamiento

- usar solo el 1% de los datos

In [2]:
file_path = 'C:\\Users\\ELI\\Downloads\\arxiv_sample.json'

# Leer todas las líneas válidas del archivo JSON
data = []
with open(file_path, 'r') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError:
            continue  # Ignorar líneas mal formateadas


# Crear el DataFrame
df = pd.DataFrame(data)

# Filtrar por los campos necesarios
df = df[['id', 'title', 'abstract']]

# Eliminar filas con valores nulos en 'title' o 'abstract'
df = df.dropna(subset=['title', 'abstract'])

# Mostrar las primeras filas para verificar
df



,id,title,abstract
0,cond-mat/0602399,Role of electron correlations in transport thr...,The transmission of correlated electrons thr...
1,1608.01034,SU(3) Polyakov linear-sigma model: bulk and sh...,Due to off-center relativistic motion of the...
2,1602.03173,Maximal Abelian gauge and a generalized BRST t...,We apply a generalized Becchi-Rouet-Stora-Ty...
3,2306.02583,Stable Diffusion is Unstable,"Recently, text-to-image models have been thr..."
4,0706.0201,A New Proof of the New Intersection Theorem,In 1987 Roberts completed the proof of the N...
...,...,...,...
27918,2109.08402,Modification of the mean free path of very hig...,Ultra-high-energy physics is about to enter ...
27919,1906.08132,Hydrodynamical response of plane correlation i...,"In high energy heavy-ion collisions, the fin..."
27920,2507.00068,MANTA: Cross-Modal Semantic Alignment and Info...,While multi-modal learning has advanced signif...
27921,2211.15410,Private Multi-Winner Voting for Machine Learning,Private multi-winner voting is the task of r...


- texto a minúsculas

In [3]:
df['title'] = df['title'].str.lower()  
df['abstract'] = df['abstract'].str.lower()

- tokenizar

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

df['title_tokens'] = df['title'].apply(tokenize)
df['abstract_tokens'] = df['abstract'].apply(tokenize)


- stopwords

In [5]:
stop_words = set(stopwords.words('english'))
punctuation_set = set(string.punctuation)

def clean_tokens(tokens):
    cleaned = []
    for token in tokens:
        token = token.lower().strip()
        # Eliminar palabras que no son puramente alfabéticas (como 's o símbolos)
        if token not in stop_words and token not in punctuation_set and re.fullmatch(r'[a-z]+', token):
            cleaned.append(token)
    return cleaned

df['title_clean'] = df['title_tokens'].apply(clean_tokens)
df['abstract_clean'] = df['abstract_tokens'].apply(clean_tokens)

In [6]:
df

,id,title,abstract,title_tokens,abstract_tokens,title_clean,abstract_clean
0,cond-mat/0602399,role of electron correlations in transport thr...,the transmission of correlated electrons thr...,"[role, of, electron, correlations, in, transpo...","[the, transmission, of, correlated, electrons,...","[role, electron, correlations, transport, doma...","[transmission, correlated, electrons, domain, ..."
1,1608.01034,su(3) polyakov linear-sigma model: bulk and sh...,due to off-center relativistic motion of the...,"[su, (, 3, ), polyakov, linear-sigma, model, :...","[due, to, off-center, relativistic, motion, of...","[su, polyakov, model, bulk, shear, viscosity, ...","[due, relativistic, motion, charged, spectator..."
2,1602.03173,maximal abelian gauge and a generalized brst t...,we apply a generalized becchi-rouet-stora-ty...,"[maximal, abelian, gauge, and, a, generalized,...","[we, apply, a, generalized, becchi-rouet-stora...","[maximal, abelian, gauge, generalized, brst, t...","[apply, generalized, brst, formulation, establ..."
3,2306.02583,stable diffusion is unstable,"recently, text-to-image models have been thr...","[stable, diffusion, is, unstable]","[recently, ,, text-to-image, models, have, bee...","[stable, diffusion, unstable]","[recently, models, thriving, despite, powerful..."
4,0706.0201,a new proof of the new intersection theorem,in 1987 roberts completed the proof of the n...,"[a, new, proof, of, the, new, intersection, th...","[in, 1987, roberts, completed, the, proof, of,...","[new, proof, new, intersection, theorem]","[roberts, completed, proof, new, intersection,..."
...,...,...,...,...,...,...,...
27918,2109.08402,modification of the mean free path of very hig...,ultra-high-energy physics is about to enter ...,"[modification, of, the, mean, free, path, of, ...","[ultra-high-energy, physics, is, about, to, en...","[modification, mean, free, path, high, energy,...","[physics, enter, new, era, thanks, impressive,..."
27919,1906.08132,hydrodynamical response of plane correlation i...,"in high energy heavy-ion collisions, the fin...","[hydrodynamical, response, of, plane, correlat...","[in, high, energy, heavy-ion, collisions, ,, t...","[hydrodynamical, response, plane, correlation,...","[high, energy, collisions, final, anisotropic,..."
27920,2507.00068,manta: cross-modal semantic alignment and info...,while multi-modal learning has advanced signif...,"[manta, :, cross-modal, semantic, alignment, a...","[while, multi-modal, learning, has, advanced, ...","[manta, semantic, alignment, optimization, mul...","[learning, advanced, significantly, current, a..."
27921,2211.15410,private multi-winner voting for machine learning,private multi-winner voting is the task of r...,"[private, multi-winner, voting, for, machine, ...","[private, multi-winner, voting, is, the, task,...","[private, voting, machine, learning]","[private, voting, task, revealing, k, binary, ..."


Indexación

- TF-IDF

In [7]:
# Unificar documentos para crear vocabulario combinado
all_docs = df['title_clean'].tolist() + df['abstract_clean'].tolist()

# Vectorizador TF-IDF (tokenizado previamente)
tfidf_vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,   
    lowercase=False,
    max_features=5000        
)

# Ajustar el vocabulario unificado
tfidf_vectorizer.fit(all_docs)
tfidf_title = tfidf_vectorizer.transform(df['title_clean'])
tfidf_abstract = tfidf_vectorizer.transform(df['abstract_clean'])

# Concatenar matrices dispersas
tfidf_combined = hstack([tfidf_title, tfidf_abstract])

# Crear nombres de columnas con prefijos
feature_names = tfidf_vectorizer.get_feature_names_out()
columns_title = [f"{name}" for name in feature_names]
columns_abstract = [f"{name}" for name in feature_names]
combined_columns = columns_title + columns_abstract

# Convertir a DataFrame disperso (sparse) para ahorro de memoria
df_tfidf = pd.DataFrame.sparse.from_spmatrix(tfidf_combined, columns=combined_columns)

df_tfidf = pd.concat([df[['id', 'title']].reset_index(drop=True), df_tfidf], axis=1)

# Mostrar primeras filas
df_tfidf

c:\Users\ELI\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,id,title,ab,abelian,abilities,ability,ablation,able,abridged,absence,...,yielding,yields,young,yr,yukawa,z,zero,zeros,zeta,zone
0,cond-mat/0602399,role of electron correlations in transport thr...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.087273,0,0,0
1,1608.01034,su(3) polyakov linear-sigma model: bulk and sh...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1602.03173,maximal abelian gauge and a generalized brst t...,0,0.498478,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2306.02583,stable diffusion is unstable,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0706.0201,a new proof of the new intersection theorem,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27918,2109.08402,modification of the mean free path of very hig...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27919,1906.08132,hydrodynamical response of plane correlation i...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27920,2507.00068,manta: cross-modal semantic alignment and info...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27921,2211.15410,private multi-winner voting for machine learning,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- BM25

In [ ]:
tokenized_documents = df['abstract_clean'].tolist()

# Crear el modelo BM25
bm25 = BM25Okapi(tokenized_documents)


- Embeddings

In [14]:
# Modelo preentrenado (puedes cambiarlo por uno más grande si lo deseas)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Unir tokens en strings
df['title_clean_str'] = df['title_clean'].apply(lambda tokens: ' '.join(tokens))
df['abstract_clean_str'] = df['abstract_clean'].apply(lambda tokens: ' '.join(tokens))

# Calcular embeddings
title_embeddings = model.encode(df['title_clean_str'].tolist(), show_progress_bar=True)
abstract_embeddings = model.encode(df['abstract_clean_str'].tolist(), show_progress_bar=True)

title_embeddings_np = np.array(title_embeddings).astype('float32')
abstract_embeddings_np = np.array(abstract_embeddings).astype('float32')

# Normalizar para usar similitud coseno
faiss.normalize_L2(title_embeddings_np)
faiss.normalize_L2(abstract_embeddings_np)


dimension = title_embeddings_np.shape[1]
index_title = faiss.IndexFlatIP(dimension)
index_title.add(title_embeddings_np)

index_abstract = faiss.IndexFlatIP(dimension)
index_abstract.add(abstract_embeddings_np)


Batches:   0%|          | 0/873 [00:00<?, ?it/s]

Batches:   0%|          | 0/873 [00:00<?, ?it/s]

Recuperación

- query

In [277]:
query = "j-multiplicity"
query_tokens = word_tokenize(query.lower())
query_tokens = [token for token in query_tokens if token not in stop_words and token not in punctuation_set]


- Búsqueda TF-IDF

In [278]:
def search_tfidf(query, top_k=10):
    query_vector = tfidf_vectorizer.transform([query])
    scores = (tfidf_abstract * query_vector.T).toarray().flatten()
    top_indices = scores.argsort()[::-1][:top_k]
    results = df.iloc[top_indices]
    return results


- Búsqueda BM25

In [279]:
def search_bm25(query, top_k=10):
    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)
    top_idx = np.argsort(scores)[::-1][:top_k]
    top_scores = np.array(scores)[top_idx]
    
    # Crear DataFrame con resultados
    results = df.iloc[top_idx][['id', 'title']].copy()
    results['score'] = top_scores
    return results


- Búsqueda FAISS

In [280]:
def search_faiss(query, top_k=10):
    query_embedding = model.encode([query], show_progress_bar=False).astype('float32')
    faiss.normalize_L2(query_embedding)
    
    D_abstract, I_abstract = index_abstract.search(query_embedding, top_k)
    
    # Construir el DataFrame de resultados
    results_abstract = df.iloc[I_abstract[0]][['id', 'title']].copy()
    results_abstract['score'] = D_abstract[0]
    
    return results_abstract.sort_values(by='score', ascending=False).reset_index(drop=True)


Mostrar resultados para cada consulta

In [281]:
print("Resultados TF-IDF:")
df_results_tfidf = search_tfidf(query)
df_results_tfidf

Resultados TF-IDF:


,id,title,abstract,title_tokens,abstract_tokens,title_clean,abstract_clean,title_clean_str,abstract_clean_str
15607,1510.08904,engel condition on enveloping algebras of lie ...,let l be a lie superalgebra over a fled of c...,"[engel, condition, on, enveloping, algebras, o...","[let, l, be, a, lie, superalgebra, over, a, fl...","[engel, condition, enveloping, algebras, lie, ...","[let, l, lie, superalgebra, fled, characterist...",engel condition enveloping algebras lie supera...,let l lie superalgebra fled characteristic env...
4995,1907.09398,"morrey spaces for schr\""odinger operators with...",let $\mathcal l=-\delta_{\mathbb h^n}+v$ be ...,"[morrey, spaces, for, schr\, '', odinger, oper...","[let, $, \mathcal, l=-\delta_, {, \mathbb, h^n...","[morrey, spaces, odinger, operators, certain, ...","[let, odinger, operator, heisenberg, group, su...",morrey spaces odinger operators certain nonneg...,let odinger operator heisenberg group sublapla...
3337,1908.07845,essential singularities of fractal zeta functions,we study the essential singularities of geom...,"[essential, singularities, of, fractal, zeta, ...","[we, study, the, essential, singularities, of,...","[essential, singularities, fractal, zeta, func...","[study, essential, singularities, geometric, z...",essential singularities fractal zeta functions,study essential singularities geometric zeta f...
25602,2202.12318,property (t) in random quotients of hyperbolic...,we study random quotients of a fixed non-ele...,"[property, (, t, ), in, random, quotients, of,...","[we, study, random, quotients, of, a, fixed, n...","[property, random, quotients, hyperbolic, grou...","[study, random, quotients, fixed, hyperbolic, ...",property random quotients hyperbolic groups de...,study random quotients fixed hyperbolic group ...
14435,1609.07181,primeval very low-mass stars and brown dwarfs....,we have conducted a search for l subdwarf ca...,"[primeval, very, low-mass, stars, and, brown, ...","[we, have, conducted, a, search, for, l, subdw...","[primeval, stars, brown, dwarfs, six, new, l, ...","[conducted, search, l, subdwarf, candidates, w...",primeval stars brown dwarfs six new l subdwarf...,conducted search l subdwarf candidates within ...
9539,cond-mat/0608460,finite-size scaling of the domain wall entropy...,the statistics of domain walls for ground st...,"[finite-size, scaling, of, the, domain, wall, ...","[the, statistics, of, domain, walls, for, grou...","[scaling, domain, wall, entropy, distributions...","[statistics, domain, walls, ground, states, is...",scaling domain wall entropy distributions j is...,statistics domain walls ground states ising sp...
9331,1201.5215,real hypersurfaces of non - flat complex plane...,the aim of the present paper is the study of...,"[real, hypersurfaces, of, non, -, flat, comple...","[the, aim, of, the, present, paper, is, the, s...","[real, hypersurfaces, non, flat, complex, plan...","[aim, present, paper, study, real, hypersurfac...",real hypersurfaces non flat complex planes ter...,aim present paper study real hypersurfaces equ...
20263,2211.11996,a class of uniformly bounded simple $\mathbb{z...,"in this paper, we classify the following sim...","[a, class, of, uniformly, bounded, simple, $, ...","[in, this, paper, ,, we, classify, the, follow...","[class, uniformly, bounded, simple, z, lie, co...","[paper, classify, following, simple, z, lie, c...",class uniformly bounded simple z lie conformal...,paper classify following simple z lie conforma...
15879,1911.03659,locally equivalent floer complexes and unorien...,we show that the local equivalence class of ...,"[locally, equivalent, floer, complexes, and, u...","[we, show, that, the, local, equivalence, clas...","[locally, equivalent, floer, complexes, unorie...","[show, local, equivalence, class, collapsed, l...",locally equivalent floer complexes unoriented ...,show local equivalence class collapsed link fl...
6505,2308.10642,frobenius trace fields of cohomologically rigi...,let $x/\mathbb

In [282]:
print("Resultados BM25:")
df_results_bm25 = search_bm25(query)
df_results_bm25


Resultados BM25:


,id,title,score
0,cond-mat/0602399,role of electron correlations in transport thr...,0.0
27922,1806.04715,cid models on real-world social networks and g...,0.0
27921,2211.15410,private multi-winner voting for machine learning,0.0
16,1911.02628,distributed mst: a smoothed analysis,0.0
17,hep-th/0511152,"conifold geometries, matrix models and quantum...",0.0
18,1304.4691,efficient calculation of determinants of symbo...,0.0
19,2202.05155,deepcent: prediction of censored event time vi...,0.0
20,cond-mat/9408076,fractal dimensions of confined clusters in two...,0.0
21,1608.05770,hierarchical scaling in systems of natural cities,0.0
22,hep-th/0004201,consistent sphere reductions and universality ...,0.0


In [283]:
print("Resultados FAISS:")
df_results_faiss = search_faiss(query)
df_results_faiss

Resultados FAISS:


,id,title,score
0,1908.02752,on maximal multiplicities for hamiltonians wit...,0.456298
1,math/0309303,a new multiplicity formula for the weyl module...,0.417946
2,1604.07801,multiplicity one for certain paramodular forms...,0.377764
3,0912.2723,singular factors of rational plane curves,0.371992
4,0806.4820,computing j-multiplicity,0.365994
5,math/0604601,invariants of singularities of pairs,0.355573
6,math/9801090,bounds on the multiplicity of eigenvalues for ...,0.348773
7,2502.10895,some formulas for epsilon multiplicity in loca...,0.347082
8,1607.01069,"demazure flags, $q$--fibonacci polynomials and...",0.346778
9,2110.12462,a degree bound for strongly nilpotent polynomi...,0.346162


Comparación de resultados tf-idf y bm25 mostrar los resultados que coincidan


In [284]:
# Comparar por ID
common_ids_tfidf_bm25 = set(df_results_tfidf['id']).intersection(df_results_bm25['id'])

# Filtrar resultados
coincidencias_tfidf_bm25 = df[df['id'].isin(common_ids_tfidf_bm25)][['id', 'title']]
display(coincidencias_tfidf_bm25)


,id,title


Comparación de resultados bm25 y faiss mostrar los resultados que coincidan

In [285]:
common_ids_bm25_faiss = set(df_results_bm25['id']).intersection(df_results_faiss['id'])
coincidencias_bm25_faiss = df[df['id'].isin(common_ids_bm25_faiss)][['id', 'title']]
display(coincidencias_bm25_faiss)


,id,title


Comparación de resultados faiss y tf-idf mostrar los resultados que coincidan

In [286]:
common_ids_faiss_tfidf = set(df_results_faiss['id']).intersection(df_results_tfidf['id'])
coincidencias_faiss_tfidf = df[df['id'].isin(common_ids_faiss_tfidf)][['id', 'title']]
display(coincidencias_faiss_tfidf)


,id,title


RAG 

top-3 documentos del índice vectorial

In [287]:
top_k = 3
top_indices = np.argsort(-results_faiss['score'])[:top_k]
top_documents = results_faiss.iloc[top_indices][['id', 'title', 'score']]
print("Top-3 documentos del índice vectorial:")
print(top_documents)

Top-3 documentos del índice vectorial:
                 id                                         title     score
0  cond-mat/9904004  electronic structure of spinel-type liv_2o_4  0.572645
1  cond-mat/0008401           electronic structure of sr_2femoo_6  0.572645
2         0806.4820                      computing j-multiplicity  0.472113


contexto

In [288]:
context_df = pd.merge(top_documents, df[['id', 'title', 'abstract']], on='id', how='left')
print("Contexto de los documentos más relevantes:")
context_df

Contexto de los documentos más relevantes:


,id,title_x,score,title_y,abstract
0,cond-mat/9904004,electronic structure of spinel-type liv_2o_4,0.572645,electronic structure of spinel-type liv_2o_4,the band structure of the cubic spinel compo...
1,cond-mat/0008401,electronic structure of sr_2femoo_6,0.572645,electronic structure of sr_2femoo_6,we have analysed the unusual electronic stru...
2,0806.4820,computing j-multiplicity,0.472113,computing j-multiplicity,the j-multiplicity is an invariant that can ...


In [289]:
# Generar texto estructurado por documento
context_text = "\n\n".join(
    f"ID de documento: {row['id']}\n"
    f"Título: {row['title_x']}\n"
    f"Puntuación de relevancia: {row['score']:.4f}\n"
    f"Abstract: {row['abstract']}"
    for _, row in context_df.iterrows()
)


In [290]:
context_text

'ID de documento: cond-mat/9904004\nTítulo: electronic structure of spinel-type liv_2o_4\nPuntuación de relevancia: 0.5726\nAbstract:   the band structure of the cubic spinel compound liv_2o_4, which has been\nreported recently to show heavy fermion behavior, has been calculated within\nthe local-density approximation using a full-potential version of the linear\naugmented-plane-wave method. the results show that partially-filled v 3d bands\nare located about 1.9 ev above the o 2p bands and the v 3d bands are split into\na lower partially-filled t_{2g} complex and an upper unoccupied e_{g} manifold.\nthe fact that the conduction electrons originate solely from the t_{2g} bands\nsuggests that the mechanism for the mass enhancement in this system is\ndifferent from that in the 4f heavy fermion systems, where these effects are\nattributed to the hybridization between the localized 4f levels and itinerant\nspd bands.\n\n\nID de documento: cond-mat/0008401\nTítulo: electronic structure of s

Prompt

In [291]:
prompt = f"""Eres una aplicación de Retrieval Augmented Generation que siempre responde en español y ademas solo responde dado el contexto. Usa el siguiente contexto para responder la pregunta.

Si la información que se pide no está presente en el contexto, responde exactamente:
"No tengo suficiente información en el contexto para responder."

Si la información está en el contexto, responde en el siguiente formato:

ID de documento: 
Título: 
Puntuación de relevancia: 
Abstract:

Contexto:
{context_text}

Pregunta:
El usuario está preguntando sobre: {query}
"""


In [ ]:
#API KEY OPENAI


In [293]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)


respuesta

In [294]:
respuesta = completion.choices[0].message.content
print("Respuesta final:")
print(respuesta)

Respuesta final:
ID de documento: 0806.4820  
Título: computing j-multiplicity  
Puntuación de relevancia: 0.4721  
Abstract:   the j-multiplicity is an invariant that can be defined for any ideal in a noetherian local ring $(r, m)$. it is equal to the hilbert-samuel multiplicity if the ideal is $m$-primary. in this paper we explore the computability of the j-multiplicity, giving another proof for the length formula and the additive formula.


Diferencias entre modelos y utilidad del RAG.


Las diferencias entre los modelos BM25, TF-IDF y FAISS radican en su enfoque para recuperar documentos relevantes. BM25 es un modelo probabilístico que considera la frecuencia de términos y la longitud del documento, mientras que TF-IDF pondera la importancia de los términos en función de su frecuencia en el documento y en el corpus. FAISS, por otro lado, utiliza embeddings vectoriales para medir similitud semántica, lo que permite capturar relaciones más complejas entre palabras y frases. El uso de RAG (Retrieval Augmented Generation) combina la recuperación de información con generación de texto, mejorando la precisión y relevancia de las respuestas al proporcionar contexto adicional.
